## API Exercise

Using the most timely and accurate data available on [https://www.census.gov/en.html](https://www.census.gov/en.html), write a python script that pulls the following poverty data for Harris County, TX in the year 2018:

- Estimate of the number of people under the age of 18 in poverty
- Estimate of the number of people of any age in poverty
- Estimate of the median household income

Your script should then write this data to a .csv file, including appropriately named headers. When finished, push your code to a public git repo.

**Requirements:**

- Use an api key to authenticate your requests
- Use the [requests](https://docs.python-requests.org/en/latest/) library (no custom libraries that pull Census data)

In [1]:
import pandas as pd
import requests
from creds import key

#### Link to poverty table for Harris county in 2018

In [2]:
url = 'https://api.census.gov/data/timeseries/poverty/saipe'
headers = {"api-key":key, "Content-Type":"application/json"}
response = requests.get(url, headers=headers)
response

<Response [200]>

In [3]:
variables = 'county:201&in=state:48&time=2018'

#### Estimate of the number of people under the age of 18 in poverty

In [4]:
stats = 'NAME,STABREV,SAEPOVU_0_17,SAEPOV0_17_PT,SAEPOVRT0_17_PT'

In [5]:
data_url = f'{url}?get={stats}&for={variables}'
response = requests.get(data_url, headers=headers)
response

<Response [200]>

In [6]:
data = response.json()
df1 = pd.DataFrame(data[1:], columns=data[0])
df1

,NAME,STABREV,SAEPOVU_0_17,SAEPOV0_17_PT,SAEPOVRT0_17_PT,time,state,county
0,Harris County,TX,1236501,306893,24.8,2018,48,201


In [7]:
print(f'There were an estimated {int(df1.SAEPOV0_17_PT[0]):,} \
people in poverty under the age of 18 in Harris County in 2018')

There were an estimated 306,893 people in poverty under the age of 18 in Harris County in 2018


#### Estimate of the number of people of any age in poverty

In [8]:
stats = 'NAME,STABREV,SAEPOVU_ALL,SAEPOVALL_PT,SAEPOVRTALL_PT'

In [9]:
data_url = f'{url}?get={stats}&for={variables}'
response = requests.get(data_url, headers=headers)
response

<Response [200]>

In [10]:
data = response.json()
df2 = pd.DataFrame(data[1:], columns=data[0])
df2

,NAME,STABREV,SAEPOVU_ALL,SAEPOVALL_PT,SAEPOVRTALL_PT,time,state,county
0,Harris County,TX,4646923,767367,16.5,2018,48,201


In [11]:
print(f'There were an estimated {int(df2.SAEPOVALL_PT[0]):,} people in poverty in Harris County in 2018')

There were an estimated 767,367 people in poverty in Harris County in 2018


#### Estimate of the median household income

In [12]:
stats = 'SAEMHI_PT'

In [13]:
data_url = f'{url}?get={stats}&for={variables}'
response = requests.get(data_url, headers=headers)
response

<Response [200]>

In [14]:
data = response.json()
df3 = pd.DataFrame(data[1:], columns=data[0])
df3

,SAEMHI_PT,time,state,county
0,60241,2018,48,201


In [15]:
print(f'The estimated median household income was ${int(df3.SAEMHI_PT[0]):,} in Harris County in 2018')

The estimated median household income was $60,241 in Harris County in 2018


#### Final Results

In [16]:
stats = 'NAME,STABREV,SAEPOVALL_PT,SAEPOV0_17_PT,SAEMHI_PT'

In [17]:
data_url = f'{url}?get={stats}&for={variables}'
response = requests.get(data_url, headers=headers)
response

<Response [200]>

In [18]:
data = response.json()
df = pd.DataFrame(data[1:], columns=data[0])
df

,NAME,STABREV,SAEPOVALL_PT,SAEPOV0_17_PT,SAEMHI_PT,time,state,county
0,Harris County,TX,767367,306893,60241,2018,48,201


In [19]:
df = df.drop(columns=['state','county'])
df = df.set_index('time').reset_index()

In [20]:
df

,time,NAME,STABREV,SAEPOVALL_PT,SAEPOV0_17_PT,SAEMHI_PT
0,2018,Harris County,TX,767367,306893,60241


In [21]:
df.columns = ['Year', 'County', 'State', 'Total Ppl in Poverty', 
              'Total Ppl under 18 in Poverty','Median Household Income']

In [22]:
df

,Year,County,State,Total Ppl in Poverty,Total Ppl under 18 in Poverty,Median Household Income
0,2018,Harris County,TX,767367,306893,60241


In [23]:
df.to_csv('harris_county_poverty_2018.csv', index=False)